In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
from autoPyTorch import AutoNetMultilabel

import os
from datetime import datetime
import sys
import json
try:
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold  # trainML
except:
    sys.path.append("../input/iterative-stratification")  # kaggle
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
BASE_PATH = os.environ.get('TRAINML_DATA_PATH') if os.environ.get('TRAINML_DATA_PATH') else '../input/lish-moa'
BASE_PATH

'/opt/data'

In [3]:
train_features = pd.read_csv(f'{BASE_PATH}/train_features.csv')
train_targets = pd.read_csv(f'{BASE_PATH}/train_targets_scored.csv')
test_features = pd.read_csv(f'{BASE_PATH}/test_features.csv')

sample_submission = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')

In [4]:
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    return df

train_data = preprocess(train_features)
test_data = preprocess(test_features)

del train_targets['sig_id']

train_targets = train_targets.loc[train_data['cp_type']==0].reset_index(drop=True)
train_data = train_data.loc[train_data['cp_type']==0].reset_index(drop=True)

In [5]:
# Data augmentation
def get_tail_labels(df: pd.DataFrame, ql=[0.03, 1.]) -> list:
    " Find the underepresented targets a.k.a. minority labels. "
    irlbl = df.sum(axis=0)
    irlbl = irlbl[(irlbl > irlbl.quantile(ql[0])) & ((irlbl < irlbl.quantile(ql[1])))]  # Filtering
    irlbl = irlbl.max() / irlbl
    threshold_irlbl = irlbl.median()
    tail_labels = irlbl[irlbl > threshold_irlbl].index.tolist()
    return tail_labels

def get_minority_samples(X: pd.DataFrame, y: pd.DataFrame, ql=[0.03, 1.]):
    " Find minority samples associated with minority labels. "
    tail_labels = get_tail_labels(y, ql=ql)
    index = y[y[tail_labels].apply(lambda x: (x == 1).any(), axis=1)].index.tolist()
    
    X_sub = X[X.index.isin(index)].reset_index(drop = True)
    y_sub = y[y.index.isin(index)].reset_index(drop = True)
    return X_sub, y_sub

def nearest_neighbour(X: pd.DataFrame, neigh) -> list:
    " Find nearest neighbors for each sample in X dataframe. "
    nbs = NearestNeighbors(n_neighbors=neigh, metric='euclidean', algorithm='kd_tree').fit(X)
    euclidean, indices = nbs.kneighbors(X)
    return indices

def MLSMOTE(X, y, n_samples, n_neighbors=5):
    " Generate new samples using MLSMOTE algorithm. "
    indices2 = nearest_neighbour(X, neigh=n_neighbors)
    n = len(indices2)
    new_X = np.zeros((n_samples, X.shape[1]))
    target = np.zeros((n_samples, y.shape[1]))
    for i in range(n_samples):
        reference = random.randint(0, n-1)
        neighbor = random.choice(indices2[reference, 1:])
        all_point = indices2[reference]
        nn_df = y[y.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)
        target[i] = np.array([1 if val > 0 else 0 for val in ser])
        ratio = random.random()
        gap = X.loc[reference,:] - X.loc[neighbor,:]
        new_X[i] = np.array(X.loc[reference,:] + ratio * gap)
    new_X = pd.DataFrame(new_X, columns=X.columns)
    target = pd.DataFrame(target, columns=y.columns)
    return new_X, target

def augment_data(X, y, oversample_args: tuple):
    " Augment feature/targets data (just doing oversampling for now)"
    n_samples, n_neighbors = oversample_args

    X_sub, y_sub = get_minority_samples(X, y)
    X_res, y_res = MLSMOTE(X_sub, y_sub, n_samples, n_neighbors)
    X_augmented = pd.concat([X, X_res])
    y_augmented = pd.concat([y, y_res])
    return X_augmented, y_augmented

In [6]:
data_oversample_args = (1000, 5)
train_data_augmented, train_targets_augmented = augment_data(train_data, train_targets, data_oversample_args)

X = train_data_augmented.values
Y = train_targets_augmented.values
X_test = test_data.values

X_original = train_data.values
Y_original = train_targets.values

In [7]:
autonet = AutoNetMultilabel(config_preset="medium_cs", result_logger_dir="logs/")
autonet.get_current_autonet_config()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


{'embeddings': ['none'],
 'lr_scheduler': ['cosine_annealing', 'plateau'],
 'networks': ['shapedresnet'],
 'preprocessors': ['none', 'truncated_svd', 'power_transformer'],
 'result_logger_dir': 'logs/',
 'log_level': 'warning',
 'hyperparameter_search_space_updates': None,
 'categorical_features': None,
 'dataset_name': None,
 'run_id': '0',
 'task_id': -1,
 'algorithm': 'bohb',
 'portfolio_type': 'greedy',
 'budget_type': 'time',
 'eta': 3,
 'min_workers': 1,
 'working_dir': '.',
 'network_interface_name': 'eth0',
 'memory_limit_mb': 1000000,
 'use_tensorboard_logger': False,
 'run_worker_on_master_node': True,
 'use_pynisher': True,
 'validation_split': 0.3,
 'refit_validation_split': 0.0,
 'cross_validator': 'none',
 'cross_validator_args': {},
 'min_budget_for_cv': 0,
 'shuffle': True,
 'imputation_strategies': ['mean', 'median', 'most_frequent'],
 'normalization_strategies': ['none', 'minmax', 'standardize', 'maxabs'],
 'over_sampling_methods': ['none'],
 'under_sampling_methods':

In [8]:
import time
start = time.time()
results_fit = autonet.fit(X_train=X,
                          Y_train=Y,
                          validation_split=0.3,
                          normalization_strategies = ['none'],
                          max_runtime=600,
                          min_budget=100,
                          max_budget=300,
                          refit=True)
end = time.time()
print("fit executed in %d seconds" % int(end - start))

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:603: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:484: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler

fit executed in 654 seconds


In [9]:
with open("logs/results_fit.json", "w") as file:
    json.dump(results_fit, file)

In [10]:
score = autonet.score(X_test=X_original, Y_test=Y_original)
print("Model accuracy score: ", score)

Model accuracy score:  0.0008201202843083652


In [12]:
preds = autonet.predict(X=X_test)

In [13]:
preds

array([[0.47449917, 0.47412273, 0.47258985, ..., 0.47503898, 0.47242862,
        0.4731424 ],
       [0.46212175, 0.46176288, 0.46127033, ..., 0.46214086, 0.46130815,
        0.46125385],
       [0.46537584, 0.46521583, 0.46429715, ..., 0.46592334, 0.46413037,
        0.4647018 ],
       ...,
       [0.46040127, 0.4602182 , 0.4598855 , ..., 0.4605803 , 0.45990843,
        0.46006277],
       [0.46608117, 0.46548915, 0.4652344 , ..., 0.46560448, 0.46527708,
        0.46552852],
       [0.46373478, 0.46328485, 0.46298856, ..., 0.46347734, 0.4630165 ,
        0.4631768 ]], dtype=float32)

In [14]:
targets = [col for col in train_targets.columns]
sample_submission[targets] = preds
sample_submission.loc[test_features['cp_type']=='ctl_vehicle', targets] = 0
sample_submission.to_csv('autonet_v1.csv', index=False)